In [2]:
#install :
!pip install python-gnupg

  Using cached python_gnupg-0.5.0-py2.py3-none-any.whl (18 kB)


In [ ]:
# note - gpg needs to be installed first:
# https://gnupg.org/download/index.html


In [3]:
import gnupg

In [4]:
gpg = gnupg.GPG()

In [5]:
#generate key
input_data = gpg.gen_key_input(
                name_email = 'me@email.com',
                passphrase= 'passphrase',
                )

#### '''what is name_email ?
 In the context of GnuPG encryption, "recipients" refers to the intended recipients of an encrypted 
message or file. When you encrypt a file using GnuPG, you can specify one or more recipients who will 
be able to decrypt the file using their private key.

In the example I provided earlier, the recipients argument of the encrypt_file() method is a list of e
mail addresses of the intended recipients. These email addresses must correspond to the email 
addresses associated with the public keys of the recipients, which must be imported into your keyring 
beforehand.

When GnuPG encrypts the file, it creates a unique session key that is used to encrypt the file's contents.
This session key is then encrypted multiple times, once for each recipient's public key. 
The resulting encrypted file can only be decrypted by someone who has the corresponding private key
for one of the recipient's public keys.'''

#### what is passphrase ?

A passphrase is a sequence of words or other text used to authenticate or secure a cryptographic key or other sensitive data. In GnuPG, a passphrase is used to protect your private key and is required in order to decrypt an encrypted file or message.

When you create a GnuPG key pair (public and private key), you are prompted to enter a passphrase that will be used to encrypt and protect your private key. This passphrase is used to unlock your private key when you want to use it to decrypt a message or file. It's important to choose a strong, unique passphrase that is not easily guessable, as it provides an additional layer of security to your encrypted data.

When you decrypt an encrypted file using GnuPG, you must enter your passphrase in order to unlock your private key and use it to decrypt the file. The passphrase is not stored anywhere, so you must remember it yourself. If you forget your passphrase, you will not be able to access your private key and will not be able to decrypt any encrypted data that was encrypted using that key.

In [6]:
key = gpg.gen_key(input_data)

In [7]:
print(key)

EB3FD2178CFC8BE26C60D47689B51D955D25D611


In [8]:
#create ascii -readable version of pub / private keys

ascii_armored_public_keys = gpg.export_keys(key.fingerprint)
ascii_armored_private_keys = gpg.export_keys(
                                            keyids=key.fingerprint,
                                            secret=True,
                                            passphrase='passphrase',)

In [9]:
#export
with open('mykeyfile' , 'w') as f:
    f.write(ascii_armored_public_keys)
    f.write(ascii_armored_private_keys)

In [10]:
# import
with open('mykeyfile') as f:
    key_data = f.read()
import_result = gpg.import_keys(key_data)

In [12]:
for k in import_result.results:
    print(k)

{'fingerprint': 'EB3FD2178CFC8BE26C60D47689B51D955D25D611', 'ok': '0', 'text': 'Not actually changed\n'}
{'fingerprint': 'EB3FD2178CFC8BE26C60D47689B51D955D25D611', 'ok': '0', 'text': 'Not actually changed\n'}
{'fingerprint': 'EB3FD2178CFC8BE26C60D47689B51D955D25D611', 'ok': '16', 'text': 'Not actually changed\nContains private key\n'}


In [16]:
##encrypt file

with open('myfile.txt','rb') as f:
    status = gpg.encrypt_file(
                                fileobj_or_path=f,
                                recipients=['me@email.com'],
                                output='myfileencrypted.txt.gpg',
                            )

In [18]:
print(status.ok)
print(status.status)
print(status.stderr)

True
encryption ok
[GNUPG:] KEY_CONSIDERED 4594959127A8BB144ABBA903BF5DFD44FAC6A8C4 0
[GNUPG:] KEY_CONSIDERED 4594959127A8BB144ABBA903BF5DFD44FAC6A8C4 2
gpg: checking the trustdb
[GNUPG:] KEY_CONSIDERED EB3FD2178CFC8BE26C60D47689B51D955D25D611 0
[GNUPG:] KEY_CONSIDERED 079337D8179C6E351E1843000FEC0FD02460D127 0
[GNUPG:] KEY_CONSIDERED 8FCEF4E18A6C73017677517825D851DE5E1CFFB6 0
[GNUPG:] KEY_CONSIDERED C0DD5F1472195E4897AF322FBF90E12797470727 0
[GNUPG:] KEY_CONSIDERED 5AA9F4AB64C5B8D7902D777DE61F9D2BBCA0E868 0
[GNUPG:] KEY_CONSIDERED 1DA7A020762E0BB1055D27EED4D65CB24C76F35D 0
[GNUPG:] KEY_CONSIDERED D85F9D5676805686BCADFFE5815FCDC263BDB321 0
[GNUPG:] KEY_CONSIDERED 930BB6CB98E42C8D86C5450AC289C86C7B79214B 0
[GNUPG:] KEY_CONSIDERED 0B6099976ECF0BC1A94D97F21A3601C19B865053 0
[GNUPG:] KEY_CONSIDERED 30F27DCDCB43B9C5C248C074B6A7A558E9F6C36D 0
[GNUPG:] KEY_CONSIDERED 8CD7B018049CD75B39BCB85DC7AA0DE61828848C 0
[GNUPG:] KEY_CONSIDERED F6E772DE0D8BC616D31F8D2C61CDE77B64BB78F0 0
[GNUPG:] KEY_CONS

In [19]:
#decrypt file

with open('myfileencrypted.txt.gpg', 'rb') as f:
    status = gpg.decrypt_file(
                fileobj_or_path=f,
                passphrase='passphrase',
                output='myfiledecrypted.txt')

In [21]:
print(status.ok)
print(status.status)
print(status.stderr)

True
decryption ok
[GNUPG:] ENC_TO 89B51D955D25D611 1 0
[GNUPG:] KEY_CONSIDERED EB3FD2178CFC8BE26C60D47689B51D955D25D611 0
gpg: encrypted with rsa2048 key, ID 89B51D955D25D611, created 2023-05-01
      "Autogenerated Key <me@email.com>"
[GNUPG:] KEY_CONSIDERED 4594959127A8BB144ABBA903BF5DFD44FAC6A8C4 0
[GNUPG:] KEY_CONSIDERED E0ADCD7CC0FB615C3B843667D5CE8E4CA48944F7 0
[GNUPG:] KEY_CONSIDERED E44FE93FCA81D132DC0CB1373BAA92CEF188763B 0
[GNUPG:] KEY_CONSIDERED 499B5DC2CB62ED54EF249C5B1713E6E869693BFF 0
[GNUPG:] KEY_CONSIDERED 69289C681705A07476280135B45D4ECD87D4CA5B 0
[GNUPG:] KEY_CONSIDERED A0C888E4936D85206F758C1E65B0B9AAFA3EB608 0
[GNUPG:] KEY_CONSIDERED 52F755221530ECD33CD0565B5DF4A7372DE804DE 0
[GNUPG:] KEY_CONSIDERED 11E8B329332A2D2BC66484E7E630010B51EE60D1 0
[GNUPG:] KEY_CONSIDERED F4512720946244DBD4D453EA94B32619C21952A8 0
[GNUPG:] KEY_CONSIDERED 6357319081EDC572BD5EEBD5145B0CF23BE2A7F1 0
[GNUPG:] KEY_CONSIDERED 1FEF002B577BF7C1032FB5C5CA1643485508073A 0
[GNUPG:] KEY_CONSIDERED 65

In [22]:
#lets see file encrypted and decrypted

In [24]:
with open('myfileencrypted.txt.gpg') as f:
    decryptedfile = f.read()

In [25]:
print(decryptedfile)

-----BEGIN PGP MESSAGE-----

hQEMA4m1HZVdJdYRAQgAkRwTH4Fwxj6qt7HraYKUYEghKE4mzu8jtyTDvAeNE/rf
RpmxGu/1SPVBKd0nML9PTAcr97riBJaFNWh9hE2mJfsZgrFs1JX8sg+Gj+eJBc5B
dAOenSdstnB/HQUm2gfzaYB0x2tMj648i4007xM03z1nTvZkYB9bFrzqwm02oM1/
3UTV7U9GCDFMFqYCXO8kgqwNnk46fUJ8Jo7zicrc3F2CV+g2UYEcJ9/gTSpoY6k/
S8ZuP2WILoo/Yxk8l7kRJE0wMOQ1x5f6iIK1JdbFR4068j/b6RL5NfYpNaF5WyMS
Q1/NEP61XUQvM53+FEFOhGfoFIKoyAEaa6wGPOKALtRTAQkCEArVIonKVmxkepRu
9eeE5yX3zX2uFxWF+FnyP2JMVNKTICqhuEQdJnjX2FWJQ838OogHcDB19ypOVxqP
eYIxLlPfmTFNX8E61ywaqYX5zyY=
=jQ3x
-----END PGP MESSAGE-----



In [26]:
with open('myfiledecrypted.txt') as f:
    myfile= f.read()

In [27]:
print(myfile)

Hello world !!
